In [10]:
from dotenv import load_dotenv

load_dotenv("../.env")

True

In [11]:
from os import getenv
from neo4j import GraphDatabase

driver = GraphDatabase.driver(
  getenv("NEO4J_HOST"),
  auth=(getenv("NEO4J_USERNAME"), getenv("NEO4J_PASSWORD"))
)

print(getenv("NEO4J_HOST"))

driver.verify_connectivity()

neo4j+s://6b0c65e8.databases.neo4j.io:7687


In [12]:

openai_api_key = getenv("OPENAI_API_KEY")

if openai_api_key != None:
  print(getenv("OPENAI_API_KEY")[:4] +"...")

sk-j...


How can we help users

1. semantic search over course content and documentation to enable better support and answer questions
2. help them to diagnose their database problems based on the solution.cypher and verify.cypher files
3. help the user with their next steps after a course


Verify a code challenge

1. Get the lesson content to get a summary of the challenge (question/{name}.adoc?)
2. Get solution.cypher
3. Get verify.cypher
4. Get the results of the verification

In [13]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(openai_api_key=openai_api_key)

llm.invoke("How are you today?")

AIMessage(content="I'm just a computer program, so I don't have feelings, but I'm here and ready to help you with anything you need! How can I assist you today?", response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 12, 'total_tokens': 47}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None}, id='run-615c40a6-75d1-4b54-9973-856eb2ee0795-0')

In [50]:
def get_lesson_content(input):
  return """<div class="module"><h1 class="module-title"><span class="module-type">Challenge</span>Registering a User</h1><article class="module-content doc"><div id="preamble">
<div class="sectionbody">
<p>Now that you have read data from the database, you are now ready to write data to the database.</p>
<p>In this challenge, you will rewrite the <code>register()</code> method in the <a href="lab/" target="_blank" rel="noopener"><code>AuthService</code></a> to do the following:</p>
<div class="olist arabic">
<ol class="arabic">
<li>
<p><a href="#_open_a_new_session">Open a new Session</a></p>
</li>
<li>
<p><a href="#_execute_a_cypher_statement_within_a_new_write_transaction">Execute a Cypher statement within a new Write Transaction</a></p>
</li>
<li>
<p><a href="#_extract_the_user_from_the_result">Extract the User from the Result</a></p>
</li>
<li>
<p><a href="#_close_the_session">Close the Session</a></p>
</li>
<li>
<p><a href="#_return_the_results">Return the Results</a></p>
</li>
</ol>
</div>
</div>
</div>
<div class="section   "><h2 id="_registering_users">Registering Users</h2><p>A key piece of functionality that the application should provide is for new users to be able to register themselves with the site.
This functionality is already built into the front end, but at the moment the credentials are hard coded in the API.
This might be fine for demo purposes, but limiting the number of users to one is bad for Neoflix’s bottom line.</p>
<p>The dummy register logic is already written into the <code>register()</code> method of the <code>AuthService</code> at <code>src/service/auth.service.js</code>.  As we can see from the snippet below, at the moment it will only accept an email address of <code>graphacademy@neo4j.com</code>.</p>
<div class="code language-js"><div class="code-header"><div class="code-language">js</div><div class="code-title">src/services/auth.service.js</div><div class="code-header-spacer"></div><button class="btn btn-copy"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M13.75 6.875V5C13.75 3.96447 12.9105 3.125 11.875 3.125H5C3.96447 3.125 3.125 3.96447 3.125 5V11.875C3.125 12.9105 3.96447 13.75 5 13.75H6.875M13.75 6.875H15C16.0355 6.875 16.875 7.71447 16.875 8.75V15C16.875 16.0355 16.0355 16.875 15 16.875H8.75C7.71447 16.875 6.875 16.0355 6.875 15V13.75M13.75 6.875H8.75C7.71447 6.875 6.875 7.71447 6.875 8.75V13.75" stroke="white" stroke-width="1.5" stroke-linecap="round" stroke-linejoin="round"></path></svg>Copy</button></div><pre class="highlight has-header"><code class="hljs language-js javascript"><span class="hljs-keyword">async</span> <span class="hljs-function"><span class="hljs-title">register</span>(<span class="hljs-params">email, plainPassword, name</span>)</span> {
  <span class="hljs-keyword">const</span> encrypted = <span class="hljs-keyword">await</span> hash(plainPassword, <span class="hljs-built_in">parseInt</span>(SALT_ROUNDS))

  <span class="hljs-comment">// <span class="hljs-doctag">TODO:</span> Handle Unique constraints in the database</span>
  <span class="hljs-keyword">if</span> (email !== <span class="hljs-string">'graphacademy@neo4j.com'</span>) {
    <span class="hljs-keyword">throw</span> <span class="hljs-keyword">new</span> ValidationError(<span class="hljs-string">`An account already exists with the email address <span class="hljs-subst">${email}</span>`</span>, {
      <span class="hljs-attr">email</span>: <span class="hljs-string">'Email address taken'</span>
    })
  }

  <span class="hljs-comment">// <span class="hljs-doctag">TODO:</span> Save user</span>

  <span class="hljs-keyword">const</span> { password, ...safeProperties } = user

  <span class="hljs-keyword">return</span> {
    ...safeProperties,
    <span class="hljs-attr">token</span>: jwt.sign(<span class="hljs-built_in">this</span>.userToClaims(safeProperties), JWT_SECRET),
  }
}</code></pre></div>
<p>From the last line, you can see that an additional <code>token</code> property is added to the return.
This represents the JWT token required to authenticate the user on any future requests.
This token is generated further down within this class.</p>
<p>You will replace these <code>TODO</code> comments with working code to complete the challenge.</p></div>
<div class="section   "><h2 id="_challenge_implementing_write_transactions">Challenge: Implementing Write Transactions</h2><p>You will follow similar steps to the previous challenge, with the one change that the Cypher statement will be executed within a Write Transaction.</p>
<p>To do so, you will need to call the <code>executeWrite()</code> method on the <code>session</code> object with a function to represent unit of work.</p>
<p>Here are the steps to complete the challenge.</p>
<a class="btn btn-lab" id="lab" target="_blank" href="./lab/">Open <code>src/services/auth.service.js</code> →</a>
<div class="section   "><h3 id="_open_a_new_session">Open a new Session</h3><p>First, open a new session:</p>
<div class="code language-js"><div class="code-header"><div class="code-language">js</div><div class="code-header-spacer"></div><button class="btn btn-copy"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M13.75 6.875V5C13.75 3.96447 12.9105 3.125 11.875 3.125H5C3.96447 3.125 3.125 3.96447 3.125 5V11.875C3.125 12.9105 3.96447 13.75 5 13.75H6.875M13.75 6.875H15C16.0355 6.875 16.875 7.71447 16.875 8.75V15C16.875 16.0355 16.0355 16.875 15 16.875H8.75C7.71447 16.875 6.875 16.0355 6.875 15V13.75M13.75 6.875H8.75C7.71447 6.875 6.875 7.71447 6.875 8.75V13.75" stroke="white" stroke-width="1.5" stroke-linecap="round" stroke-linejoin="round"></path></svg>Copy</button></div><pre class="highlight has-header"><code class="hljs language-js javascript"><span class="hljs-comment">// Open a new session</span>
<span class="hljs-keyword">const</span> session = <span class="hljs-built_in">this</span>.driver.session()</code></pre></div></div>
<div class="section   "><h3 id="_execute_a_cypher_statement_within_a_new_write_transaction">Execute a Cypher statement within a new Write Transaction</h3><p>Next, within that session, run the <code>executeWrite()</code> method with two arguments:</p>
<div class="olist arabic">
<ol class="arabic">
<li>
<p>A Cypher statement to <code>CREATE</code> a new <code>:User</code> node in the database, with parameters passed to the function prefixed with a dollar sign (<code>$</code>).</p>
</li>
<li>
<p>An object containing the values relating to the parameters prefixed with <code>$</code> in the Cypher statement.</p>
<div class="olist loweralpha">
<ol class="loweralpha" type="a">
<li>
<p><code>email</code> - provided as the first parameter to the <code>.register()</code> method</p>
</li>
<li>
<p><code>encrypted</code> - The encrypted password</p>
</li>
<li>
<p><code>name</code> - The name as provided as the third parameter to the <code>.register()</code> method</p>
</li>
</ol>
</div>
</li>
</ol>
</div>
<div class="code language-js"><div class="code-header"><div class="code-language">js</div><div class="code-header-spacer"></div><button class="btn btn-copy"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M13.75 6.875V5C13.75 3.96447 12.9105 3.125 11.875 3.125H5C3.96447 3.125 3.125 3.96447 3.125 5V11.875C3.125 12.9105 3.96447 13.75 5 13.75H6.875M13.75 6.875H15C16.0355 6.875 16.875 7.71447 16.875 8.75V15C16.875 16.0355 16.0355 16.875 15 16.875H8.75C7.71447 16.875 6.875 16.0355 6.875 15V13.75M13.75 6.875H8.75C7.71447 6.875 6.875 7.71447 6.875 8.75V13.75" stroke="white" stroke-width="1.5" stroke-linecap="round" stroke-linejoin="round"></path></svg>Copy</button></div><pre class="highlight has-header"><code class="hljs language-js javascript"><span class="hljs-comment">// Create the User node in a write transaction</span>
<span class="hljs-keyword">const</span> res = <span class="hljs-keyword">await</span> session.executeWrite(
  <span class="hljs-function"><span class="hljs-params">tx</span> =&gt;</span> tx.run(
    <span class="hljs-string">`
      CREATE (u:User {
        userId: randomUuid(),
        email: $email,
        password: $encrypted,
        name: $name
      })
      RETURN u
    `</span>,
    { email, encrypted, name }
  )
)</code></pre></div></div>
<div class="section   "><h3 id="_extract_the_user_from_the_result">Extract the User from the Result</h3><p>The Cypher statement executed above returns the newly-created <code>:User</code> node as <code>u</code>.
As this query creates a single node, it will only ever return one result, so the <code>u</code> value may be taken from the first row.</p>
<div class="code language-js"><div class="code-header"><div class="code-language">js</div><div class="code-header-spacer"></div><button class="btn btn-copy"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M13.75 6.875V5C13.75 3.96447 12.9105 3.125 11.875 3.125H5C3.96447 3.125 3.125 3.96447 3.125 5V11.875C3.125 12.9105 3.96447 13.75 5 13.75H6.875M13.75 6.875H15C16.0355 6.875 16.875 7.71447 16.875 8.75V15C16.875 16.0355 16.0355 16.875 15 16.875H8.75C7.71447 16.875 6.875 16.0355 6.875 15V13.75M13.75 6.875H8.75C7.71447 6.875 6.875 7.71447 6.875 8.75V13.75" stroke="white" stroke-width="1.5" stroke-linecap="round" stroke-linejoin="round"></path></svg>Copy</button></div><pre class="highlight has-header"><code class="hljs language-js javascript"><span class="hljs-comment">// Extract the user from the result</span>
<span class="hljs-keyword">const</span> [ first ] = res.records
<span class="hljs-keyword">const</span> node = first.get(<span class="hljs-string">'u'</span>)

<span class="hljs-keyword">const</span> { password, ...safeProperties } = node.properties</code></pre></div>
<p>The final line uses the <a href="https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Operators/Destructuring_assignment" target="_blank" rel="noopener">Destructuring assignment</a> and <a href="https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Operators/Spread_syntax" target="_blank" rel="noopener">Spread syntax (<code>…​</code>)</a> to extract a set of <em>safe</em> properties that can be used in the JWT token.</p></div>
<div class="section   "><h3 id="_close_the_session">Close the Session</h3><p>Before the user information is returned, make sure that the session is closed.</p>
<div class="code language-js"><div class="code-header"><div class="code-language">js</div><div class="code-header-spacer"></div><button class="btn btn-copy"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M13.75 6.875V5C13.75 3.96447 12.9105 3.125 11.875 3.125H5C3.96447 3.125 3.125 3.96447 3.125 5V11.875C3.125 12.9105 3.96447 13.75 5 13.75H6.875M13.75 6.875H15C16.0355 6.875 16.875 7.71447 16.875 8.75V15C16.875 16.0355 16.0355 16.875 15 16.875H8.75C7.71447 16.875 6.875 16.0355 6.875 15V13.75M13.75 6.875H8.75C7.71447 6.875 6.875 7.71447 6.875 8.75V13.75" stroke="white" stroke-width="1.5" stroke-linecap="round" stroke-linejoin="round"></path></svg>Copy</button></div><pre class="highlight has-header"><code class="hljs language-js javascript"><span class="hljs-comment">// Close the session</span>
<span class="hljs-keyword">await</span> session.close()</code></pre></div></div>
<div class="section   "><h3 id="_return_the_results">Return the Results</h3><p>The return statement has already been written, so this can be left as it is.</p>
<div class="code language-js"><div class="code-header"><div class="code-language">js</div><div class="code-header-spacer"></div><button class="btn btn-copy"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M13.75 6.875V5C13.75 3.96447 12.9105 3.125 11.875 3.125H5C3.96447 3.125 3.125 3.96447 3.125 5V11.875C3.125 12.9105 3.96447 13.75 5 13.75H6.875M13.75 6.875H15C16.0355 6.875 16.875 7.71447 16.875 8.75V15C16.875 16.0355 16.0355 16.875 15 16.875H8.75C7.71447 16.875 6.875 16.0355 6.875 15V13.75M13.75 6.875H8.75C7.71447 6.875 6.875 7.71447 6.875 8.75V13.75" stroke="white" stroke-width="1.5" stroke-linecap="round" stroke-linejoin="round"></path></svg>Copy</button></div><pre class="highlight has-header"><code class="hljs language-js javascript"><span class="hljs-keyword">return</span> {
  ...safeProperties,
  <span class="hljs-attr">token</span>: jwt.sign(<span class="hljs-built_in">this</span>.userToClaims(safeProperties), JWT_SECRET),
}</code></pre></div></div></div>
<div class="section   "><h2 id="_working_solution">Working Solution</h2><details>
<summary class="title">Click here to reveal the fully-implemented <code>register()</code> method.</summary>
<div class="content">
<div class="code language-js"><div class="code-header"><div class="code-language">js</div><div class="code-header-spacer"></div><button class="btn btn-copy"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M13.75 6.875V5C13.75 3.96447 12.9105 3.125 11.875 3.125H5C3.96447 3.125 3.125 3.96447 3.125 5V11.875C3.125 12.9105 3.96447 13.75 5 13.75H6.875M13.75 6.875H15C16.0355 6.875 16.875 7.71447 16.875 8.75V15C16.875 16.0355 16.0355 16.875 15 16.875H8.75C7.71447 16.875 6.875 16.0355 6.875 15V13.75M13.75 6.875H8.75C7.71447 6.875 6.875 7.71447 6.875 8.75V13.75" stroke="white" stroke-width="1.5" stroke-linecap="round" stroke-linejoin="round"></path></svg>Copy</button></div><pre class="highlight has-header"><code class="hljs language-js javascript"><span class="hljs-keyword">async</span> <span class="hljs-function"><span class="hljs-title">register</span>(<span class="hljs-params">email, plainPassword, name</span>)</span> {
  <span class="hljs-keyword">const</span> encrypted = <span class="hljs-keyword">await</span> hash(plainPassword, <span class="hljs-built_in">parseInt</span>(SALT_ROUNDS))

  <span class="hljs-comment">// <span class="hljs-doctag">TODO:</span> Handle Unique constraints in the database</span>
  <span class="hljs-keyword">if</span> (email !== <span class="hljs-string">'graphacademy@neo4j.com'</span>) {
    <span class="hljs-keyword">throw</span> <span class="hljs-keyword">new</span> ValidationError(<span class="hljs-string">`An account already exists with the email address <span class="hljs-subst">${email}</span>`</span>, {
      <span class="hljs-attr">email</span>: <span class="hljs-string">'Email address taken'</span>
    })
  }

  <span class="hljs-comment">// Open a new session</span>
  <span class="hljs-keyword">const</span> session = <span class="hljs-built_in">this</span>.driver.session()

  <span class="hljs-comment">// Create the User node in a write transaction</span>
  <span class="hljs-keyword">const</span> res = <span class="hljs-keyword">await</span> session.executeWrite(
    <span class="hljs-function"><span class="hljs-params">tx</span> =&gt;</span> tx.run(
      <span class="hljs-string">`
        CREATE (u:User {
          userId: randomUuid(),
          email: $email,
          password: $encrypted,
          name: $name
        })
        RETURN u
      `</span>,
      { email, encrypted, name }
    )
  )

  <span class="hljs-comment">// Extract the user from the result</span>
  <span class="hljs-keyword">const</span> [ first ] = res.records
  <span class="hljs-keyword">const</span> node = first.get(<span class="hljs-string">'u'</span>)

  <span class="hljs-keyword">const</span> { password, ...safeProperties } = node.properties

  <span class="hljs-comment">// Close the session</span>
  <span class="hljs-keyword">await</span> session.close()

  <span class="hljs-keyword">return</span> {
    ...safeProperties,
    <span class="hljs-attr">token</span>: jwt.sign(<span class="hljs-built_in">this</span>.userToClaims(safeProperties), JWT_SECRET),
  }
}</code></pre></div>
</div>
</details></div>
<div class="section   "><h2 id="_testing">Testing</h2><p>To test that this functionality has been correctly implemented, run the following code in a new terminal session:</p>
<div class="code language-sh"><div class="code-header"><div class="code-language">sh</div><div class="code-title">Running the test</div><div class="code-header-spacer"></div><button class="btn btn-copy"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M13.75 6.875V5C13.75 3.96447 12.9105 3.125 11.875 3.125H5C3.96447 3.125 3.125 3.96447 3.125 5V11.875C3.125 12.9105 3.96447 13.75 5 13.75H6.875M13.75 6.875H15C16.0355 6.875 16.875 7.71447 16.875 8.75V15C16.875 16.0355 16.0355 16.875 15 16.875H8.75C7.71447 16.875 6.875 16.0355 6.875 15V13.75M13.75 6.875H8.75C7.71447 6.875 6.875 7.71447 6.875 8.75V13.75" stroke="white" stroke-width="1.5" stroke-linecap="round" stroke-linejoin="round"></path></svg>Copy</button></div><pre class="highlight has-header"><code class="hljs language-sh bash">npm run <span class="hljs-built_in">test</span> 03</code></pre></div>
<p>The test file is located at <a href="https://github.com/neo4j-graphacademy/app-nodejs/blob/main/test/challenges/03-registering-a-user.spec.js" target="_blank" rel="noopener"><code>test/challenges/03-registering-a-user.spec.js</code></a>.</p>
<details class="stuck">
<summary class="title">Are you stuck?  Click here for help</summary>
<div class="content">
<p>If you get stuck, you can see a working solution by checking out the <code>03-registering-a-user</code> branch by running:</p>
<div class="code language-sh"><div class="code-header"><div class="code-language">sh</div><div class="code-title">Check out the 03-registering-a-user branch</div><div class="code-header-spacer"></div><button class="btn btn-copy"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M13.75 6.875V5C13.75 3.96447 12.9105 3.125 11.875 3.125H5C3.96447 3.125 3.125 3.96447 3.125 5V11.875C3.125 12.9105 3.96447 13.75 5 13.75H6.875M13.75 6.875H15C16.0355 6.875 16.875 7.71447 16.875 8.75V15C16.875 16.0355 16.0355 16.875 15 16.875H8.75C7.71447 16.875 6.875 16.0355 6.875 15V13.75M13.75 6.875H8.75C7.71447 6.875 6.875 7.71447 6.875 8.75V13.75" stroke="white" stroke-width="1.5" stroke-linecap="round" stroke-linejoin="round"></path></svg>Copy</button></div><pre class="highlight has-header"><code class="hljs language-sh bash">git checkout 03-registering-a-user</code></pre></div>
<p>You may have to commit or stash your changes before checking out this branch.  You can also <a href="#" class="classroom-support-toggle">click here to expand the Support pane</a>.</p>
</div>
</details></div>
<div class="section  verify "><h2 id="_verifying_the_test">Verifying the Test</h2><p>Here is where things get interesting.</p>
<p>If you have completed the course to this point, you should have a project that connects to the Neo4j Sandbox instance.</p>
<p>If the test above has succeeded, there should be a <code>:User</code> node in the sandbox with the email address <code>graphacademy@neo4j.com</code>, name <code>Graph Academy</code> and an encrypted password.</p>
                <button class="btn btn--primary btn-submit btn-verify">
                    <span class="loading-indicator"><svg width="12" height="12" viewBox="0 0 12 12" fill="none" xmlns="http://www.w3.org/2000/svg">
    <path d="M6 3.25C6.41421 3.25 6.75 2.91421 6.75 2.5V1C6.75 0.585786 6.41421 0.25 6 0.25C5.58579 0.25 5.25 0.585786 5.25 1V2.5C5.25 2.91421 5.58579 3.25 6 3.25Z" fill="black"></path>
    <path d="M4.13406 3.05305L3.07306 1.99255C2.77589 1.71033 2.30973 1.71033 2.01256 1.99255C1.71977 2.28543 1.71977 2.76018 2.01256 3.05305L3.07356 4.11355C3.21405 4.25447 3.40507 4.33333 3.60406 4.33255C3.90698 4.33208 4.17988 4.14943 4.29581 3.86957C4.41173 3.58971 4.34792 3.26758 4.13406 3.05305Z" fill="black"></path>
    <path d="M0.25 6C0.25 6.41421 0.585786 6.75 1 6.75H2.5C2.91421 6.75 3.25 6.41421 3.25 6C3.25 5.58579 2.91421 5.25 2.5 5.25H1C0.585786 5.25 0.25 5.58579 0.25 6Z" fill="black"></path>
    <path d="M3.47867 7.76202C3.27984 7.76127 3.08898 7.84013 2.94867 7.98102L1.88767 9.04202C1.6092 9.33763 1.61662 9.80118 1.90439 10.0877C2.19217 10.3743 2.65575 10.3797 2.95017 10.1L4.01117 9.03902C4.2251 8.82442 4.28888 8.50218 4.17282 8.22227C4.05677 7.94236 3.78368 7.75979 3.48067 7.75952L3.47867 7.76202Z" fill="black"></path>
    <path d="M5.25 11C5.25 11.4142 5.58579 11.75 6 11.75C6.41421 11.75 6.75 11.4142 6.75 11V9.5C6.75 9.08579 6.41421 8.75 6 8.75C5.58579 8.75 5.25 9.08579 5.25 9.5V11Z" fill="black"></path>
    <path d="M8.02428 8.00297C7.73149 8.29585 7.73149 8.7706 8.02428 9.06347L9.08528 10.124C9.3784 10.4179 9.85432 10.4186 10.1483 10.1255C10.4422 9.83235 10.4429 9.35643 10.1498 9.06247L9.08628 7.99997C8.94517 7.85931 8.7539 7.78057 8.55465 7.78113C8.35541 7.78169 8.16458 7.86151 8.02428 8.00297Z" fill="black"></path>
    <path d="M11.75 6C11.75 5.58579 11.4142 5.25 11 5.25H9.5C9.08579 5.25 8.75 5.58579 8.75 6C8.75 6.41421 9.08579 6.75 9.5 6.75H11C11.4142 6.75 11.75 6.41421 11.75 6Z" fill="black"></path>
    <path d="M9.52103 1.70496C9.32219 1.70421 9.13133 1.78307 8.99103 1.92396L7.93103 2.98396C7.79012 3.12465 7.71094 3.31559 7.71094 3.51471C7.71094 3.71383 7.79012 3.90478 7.93103 4.04546C8.2239 4.33825 8.69865 4.33825 8.99153 4.04546L10.052 2.98546C10.2667 2.77089 10.3309 2.44806 10.2147 2.16767C10.0984 1.88728 9.82457 1.70462 9.52103 1.70496Z" fill="black"></path>
    </svg></span>
                    <span class="btn-label">Check Database</span>
                </button>
<button class="admonition-show-hint">Show Hint</button><div class="admonition admonition--tip hint"><p class="admonition-title">Hint</p><p>You can run the following query to check for the user within the database.
If the <code>shouldVerify</code> value returns true, the verification should be successful.</p>
<div class="code language-cypher"><div class="code-header"><div class="code-language">cypher</div><div class="code-header-spacer"></div><button class="btn btn-copy"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M13.75 6.875V5C13.75 3.96447 12.9105 3.125 11.875 3.125H5C3.96447 3.125 3.125 3.96447 3.125 5V11.875C3.125 12.9105 3.96447 13.75 5 13.75H6.875M13.75 6.875H15C16.0355 6.875 16.875 7.71447 16.875 8.75V15C16.875 16.0355 16.0355 16.875 15 16.875H8.75C7.71447 16.875 6.875 16.0355 6.875 15V13.75M13.75 6.875H8.75C7.71447 6.875 6.875 7.71447 6.875 8.75V13.75" stroke="white" stroke-width="1.5" stroke-linecap="round" stroke-linejoin="round"></path></svg>Copy</button><button class="btn btn-play"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M4.375 4.71056C4.375 3.9975 5.1395 3.54547 5.7643 3.88911L15.3815 9.17855C16.0291 9.53473 16.0291 10.4653 15.3815 10.8215L5.7643 16.1109C5.1395 16.4545 4.375 16.0025 4.375 15.2894V4.71056Z" stroke="white" stroke-width="1.5" stroke-linecap="round" stroke-linejoin="round"></path></svg>Run</button></div><pre class="highlight has-header"><code class="hljs language-cypher"><span class="hljs-keyword">MATCH</span> (u:<span class="hljs-type">User</span> {email: <span class="hljs-string">'graphacademy@neo4j.com'</span>})
<span class="hljs-keyword">RETURN</span> u.email, u.name, u.password,
    (u.email = <span class="hljs-string">'graphacademy@neo4j.com'</span> AND u.name = <span class="hljs-string">'Graph Academy'</span> AND u.password &lt;&gt; <span class="hljs-string">'letmein'</span>) <span class="hljs-keyword">AS</span> shouldVerify</code></pre></div></div>
<button class="admonition-show-solution">Show Solution</button><div class="admonition admonition--tip solution"><p class="admonition-title">Solution</p><p>The following statement will mimic the behaviour of the test, merging a new <code>:User</code> node with the email address <code>graphacademy@neo4j.com</code> and assigning a random UUID value to the <code>.userId</code> property.</p>
<div class="code language-cypher"><div class="code-header"><div class="code-language">cypher</div><div class="code-header-spacer"></div><button class="btn btn-copy"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M13.75 6.875V5C13.75 3.96447 12.9105 3.125 11.875 3.125H5C3.96447 3.125 3.125 3.96447 3.125 5V11.875C3.125 12.9105 3.96447 13.75 5 13.75H6.875M13.75 6.875H15C16.0355 6.875 16.875 7.71447 16.875 8.75V15C16.875 16.0355 16.0355 16.875 15 16.875H8.75C7.71447 16.875 6.875 16.0355 6.875 15V13.75M13.75 6.875H8.75C7.71447 6.875 6.875 7.71447 6.875 8.75V13.75" stroke="white" stroke-width="1.5" stroke-linecap="round" stroke-linejoin="round"></path></svg>Copy</button><button class="btn btn-play"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M4.375 4.71056C4.375 3.9975 5.1395 3.54547 5.7643 3.88911L15.3815 9.17855C16.0291 9.53473 16.0291 10.4653 15.3815 10.8215L5.7643 16.1109C5.1395 16.4545 4.375 16.0025 4.375 15.2894V4.71056Z" stroke="white" stroke-width="1.5" stroke-linecap="round" stroke-linejoin="round"></path></svg>Run</button></div><pre class="highlight has-header"><code class="hljs language-cypher"><span class="hljs-keyword">MERGE</span> (u:<span class="hljs-type">User</span> {email: <span class="hljs-string">"graphacademy@neo4j.com"</span>})
<span class="hljs-keyword">SET</span> u.userId =<span class="hljs-functionCall"> randomUuid()</span>,
    u.createdAt =<span class="hljs-functionCall"> datetime()</span>,
    u.authenticatedAt =<span class="hljs-functionCall"> datetime()</span></code></pre></div>
<p>Once you have run this statement, click <strong>Try again…​</strong>* to complete the challenge.</p></div></div>
<div class="section  summary "><h2 id="_lesson_summary">Lesson Summary</h2><p>In this Challenge, you wrote the code to create a new User node to Neo4j.</p>
<p>We still have a <code>TODO</code> comment in this method for handling unique constraint violations in the database, so let’s learn about that in the next lesson.</p></div></article></div>
"""

def get_last_attempt(input):
  return [
    {"filename": "verify.adoc", "answers": [False]}
  ]

def get_verify_output(input):
  return [
    {"outcome": True, "reason": 'A (:User) node must exist with the email property set to "graphacademy@neo4j.com"'},
    {"outcome": False, "reason": 'The password property should be encrypted'},
    {"outcome": False, "reason": 'The name property should equal "Graph Academy"'},
  ]

def get_verify_cypher(input):
  return """ OPTIONAL MATCH (u:User {email: 'graphacademy@neo4j.com'})
UNWIND [
    [ u IS NOT NULL, 'A (:User) node must exist with the email property set to "graphacademy@neo4j.com"' ],
    [ u.name = 'GraphAcademy', 'The name property should equal "Graph Academy"' ],
    [ u.password != 'letmein', 'The password property should be encrypted' ],
] AS condition

RETURN condition[0] AS outcome, condition[1] AS reason

  """

def get_solution_cypher(input):
  return """OPTIONAL MATCH (u:User {email: 'graphacademy@neo4j.com'})
UNWIND [
    [ u IS NOT NULL, 'A (:User) node must exist with the email property set to "graphacademy@neo4j.com"' ],
    [ u.name = 'GraphAcademy', 'The name property should equal "Graph Academy"' ],
    [ u.password != 'letmein', 'The password property should be encrypted' ],
] AS condition

RETURN condition[0] AS outcome, condition[1] AS reason
"""


In [52]:
# from langchain.output_parsers.string import StrOutputParser
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

inspect_prompt = ChatPromptTemplate.from_messages([
  (
    "system",
    """
      You are an expert Neo4j developer inspecting a database for a user.
      The user is trying to pass a lesson on GraphAcademy, where they need to complete a hands-on challenge.

      Here is the lesson content and the solution in HTML format:
      ```
      {lesson_content}
      ```

      Here is the verification script:
      ```
      {verify_cypher}
      ```

      The following output has been returned by the verification script:

      ``
      {verify_output}
      ```

      Provide simple instructions to help the user fix the issue.
      Only reference the rows that have failed the verification.
      Show code samples where possible.
    """
    )
])

inspect_chain = (
  {
    "lesson_content": get_lesson_content,
    "attempt": get_last_attempt,
    "solution_cypher": get_solution_cypher,
    "verify_cypher": get_verify_cypher,
    "verify_output": get_verify_output,
  }
  | inspect_prompt
  | llm
  | StrOutputParser()
)


In [54]:
response = inspect_chain.invoke({})

from IPython.display import display, Markdown
display(Markdown(response))

It looks like the verification script has identified two issues that need to be fixed:

1. The password property should be encrypted.
2. The name property should equal "Graph Academy".

To fix these issues, you can update the code in the `register()` method of the `AuthService` class in `src/service/auth.service.js`.

1. **Encrypt the password**:
   - In the `register()` method, make sure that the `plainPassword` is encrypted before being saved to the database. You can use the `hash()` function with `SALT_ROUNDS` to hash the password.
   
   Here is an example of how to hash the password:
   ```javascript
   const encrypted = await hash(plainPassword, parseInt(SALT_ROUNDS));
   ```

2. **Update the name property**:
   - Ensure that the `name` property matches the expected value of "Graph Academy". Check that the `name` parameter passed to the function is correctly assigned to the `name` property of the user node.

After making these changes, you can rerun the verification script to ensure that the issues have been resolved.